### 使用聊天模型和提示模板构建一个简单的 LLM 应用程序

在本快速入门教程中，我们将向你展示如何使用 LangChain 构建一个简单的大型语言模型（LLM）应用程序。该应用程序将把文本从英文翻译成另一种语言。

这是一个相对简单的 LLM 应用程序 —— 它只包含一次 LLM 调用和一些提示词（prompting）操作。尽管如此，这是开始学习 LangChain 的绝佳方式 —— 许多功能只需通过一些提示词和一次 LLM 调用就可以实现！

阅读完本教程后，你将对以下内容有一个高层次的了解：

- 如何使用语言模型  
- 如何使用提示模板（prompt templates）  

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

### 使用语言模型

首先，我们来学习如何单独使用一个语言模型。LangChain 支持多种不同的语言模型，并且你可以根据需要灵活地切换使用。

有关如何开始使用特定模型的详细信息，请参考 LangChain 的[**支持的集成列表**](https://python.langchain.com/docs/integrations/chat/)。



In [2]:
from langchain_community.chat_models.tongyi import ChatTongyi

model = ChatTongyi(
    streaming=True,
)

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("以下内容从英文翻译成地道的粤语"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='哈囉！😄', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '5035d078-03ed-9efa-b3bd-685af1d23184', 'token_usage': {'input_tokens': 25, 'output_tokens': 5, 'total_tokens': 30, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--9812a6d4-075f-4174-838c-0ad8286e797b-0')

- 请注意，ChatModels 接收消息对象作为输入，并生成消息对象作为输出。
- 除了文本内容外，消息对象还传达对话角色并保存重要数据，例如工具调用次数和代币使用次数。
- LangChain 还支持通过字符串或 OpenAI 格式输入聊天模型。以下代码等效：

In [4]:
res1 = model.invoke("Hello")

res2 = model.invoke([{"role": "user", "content": "Hello"}])

res3 = model.invoke([HumanMessage("Hello")])

In [5]:
res1

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'f9bf7853-e704-940e-a11e-85a80144663e', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--442f970a-f5c0-4586-b474-22b7156bb162-0')

In [6]:
res2

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '88ef4c29-b6bd-9f40-b3ce-94b3e561aeea', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--6a4845a4-905d-4816-91c3-2a4f56347fdf-0')

In [7]:
res3

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '7e6be60a-9184-9095-8555-2c084cd2c689', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--83df58a3-c9e8-446d-8196-a9ad4be4a982-0')

**流式传输（Streaming）**  
由于聊天模型是可运行对象（Runnables），它们提供了一个标准接口，其中包括异步和流式调用模式。这使我们能够从聊天模型中逐个地流式传输（逐步接收）响应的 token（即生成的文字片段）。

In [8]:
for token in model.stream(messages):
    print(token.content, end="|")

哈|囉|！😊||

## 提示模板
目前，我们是将一组消息直接传递给语言模型。那么，这组消息是从哪里来的呢？  
通常，它是由用户输入和应用程序逻辑共同构建而成的。这种应用程序逻辑通常会接收用户的原始输入，并将其转换为一个可用于语言模型的消息列表。  
常见的转换操作包括添加系统消息（system message）或将用户输入填入预定义的模板中。

**提示模板（Prompt Templates）** 是 LangChain 中的一个概念，专门用于辅助这种转换。  
它可以接收用户的原始输入，并返回一个已经格式化好的提示信息（prompt），这个提示可以直接传给语言模型使用。

下面我们来创建一个提示模板。该模板将接收两个用户变量：

- `language`：要翻译成的目标语言  
- `text`：需要翻译的文本

### 系统提示词 - system prompt
 - system prompt 每次运行都不变,是开发者提前设定好的（但是可以传参）。
 - 每次都放在user prompt之前。
 - 并且优先级比user prompt高
### 用户提示词 - user prompt
 - user prompt 用户输入的内容，每次调用都会不一样。

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from Chinese Mandarin into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

请注意，`ChatPromptTemplate` 支持在单个模板中包含多种消息角色（message roles）。  
我们将 `language` 参数格式化到系统消息（system message）中，将用户的文本（user text）格式化到用户消息（user message）中。

这个提示模板的输入是一个字典。我们可以单独尝试这个提示模板，看看它的输出是什么样子的。

In [10]:
prompt = prompt_template.invoke({"language": "粤语", "text": "你吃了吗!"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from Chinese Mandarin into 粤语', additional_kwargs={}, response_metadata={}), HumanMessage(content='你吃了吗!', additional_kwargs={}, response_metadata={})])

我们可以看到，它返回了一个包含两条消息的 `ChatPromptValue`。如果我们想要直接访问这些消息，可以这样做：

In [11]:
prompt.to_messages()

[SystemMessage(content='Translate the following from Chinese Mandarin into 粤语', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='你吃了吗!', additional_kwargs={}, response_metadata={})]

In [12]:
response = model.invoke(prompt)
print(response.content)

你食咗未呀！


### 总结

就这样！在本教程中，你学习了如何构建你的第一个简单的大型语言模型（LLM）应用程序。你了解了如何使用语言模型、如何创建提示模板（prompt template）。

这仅仅是你成为一名熟练的 AI 工程师所需知识的冰山一角。幸运的是——我们还有大量其他学习资源！